#[문제]
- law_2.docx 파일을 읽고, chroma 저장  
- LLM질문 -> 답변  
- 전세사기피해에 관한 법률 질문만 받기.
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma #로컬
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders import Docx2txtLoader ## 도큐먼트 문서를 텍스트로 읽어옴

In [4]:
## RAG 
## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx') #인스턴스 생성 #문서 확인

text_splitter = RecursiveCharacterTextSplitter(  #분할을 하기위한 설정
    chunk_size=1500,
    chunk_overlap=200,
) #나중에 여러번의 질문을 통해 정확도를 확인

document_list = loader.load_and_split(text_splitter=text_splitter) #파일을 갖고 파라미터에 설정된 읽고 분할
document_list


[Document(metadata={'source': 'law_2.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또는 신고 건별로 각각 제1호가목에 따라 계산한 금액\n\n  ② 제1항제1호가

In [5]:
len(document_list)

5

In [ ]:

## 2. 임베딩 -> 벡터 데이터베이스에 저장 

## 2.1. 환경변수 읽어오기
load_dotenv() #잘 읽어왔다는 True


## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large') #성능 좋은 유료  # 위에서 OPENAI_API_KEY 읽었기 때문에 
embedding


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000022FBF37AE90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000022FBF399A80>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [10]:
# # ## 2.3. 벡터 데이터베이스에 저장 ########한번 실행했기 때문에
##[방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

## 로컬에 저장된 임베딩 데이터 가져오기

database =Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database


In [107]:
## 3. 질문이 있으면, 벡터 데이터베이스(vector_store)에서 유사도 검색

## 3.1. 사용자 질문
query = '오늘 서울 날씨는?'
# query = '전세사기 피해자는 누구인가요? '

## 3.2. 벡터 데이터베이스에서 유사도 검색 (점수 포함)
retrieved_docs = database.similarity_search(query=query, k=2)
retrieved_docs


[Document(id='ff10f339-e56d-4780-a1d5-b8f1c6c3ec42', metadata={'source': 'law_2.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또

In [20]:
len(retrieved_docs)

2

In [ ]:
a = '!'
a += '가'
a

'!가'

In [92]:
## 문서 객체에서 문자열만 추출
## [방법 1]
ex = ''

for doc in retrieved_docs:
    # print(doc.page_content + '🐱‍💻🐱‍💻\n')
    ex += doc.page_content + '💖💖\n\n'

ex

'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또는 신고 건별로 각각 제1호가목에 따라 계산한 금액\n\n  ② 제1항제1호가목의 계산식에서 “전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액”은 법 제23조제3항 전단에

In [101]:
## 문자열 : join 함수
'$$'.join(['one','two','three'])

'one$$two$$three'

In [ ]:
context = '🤷‍♀️🤷‍♀️\n\n'.join([doc.page_content for doc in retrieved_docs])
context

'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또는 신고 건별로 각각 제1호가목에 따라 계산한 금액\n\n  ② 제1항제1호가목의 계산식에서 “전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액”은 법 제23조제3항 전단에

In [3]:
var = (num for num in ['a', 'b'])
var

<generator object <genexpr> at 0x000001D6EB6B9BD0>

In [4]:
type(var)

generator

In [5]:
tu = ('a', 'b')
type(tu)

tuple

In [ ]:
'🤣'.join(num for num in ['a', 'b'])


'a🤣b'

In [2]:

'🤣'.join([num for num in ['a', 'b']])

'a🤣b'

In [108]:
## [방법 2]
## 리스트 for 문
context = [ doc.page_content+'🤷‍♀️🤷‍♀️' for doc in retrieved_docs]
context

['전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또는 신고 건별로 각각 제1호가목에 따라 계산한 금액\n\n  ② 제1항제1호가목의 계산식에서 “전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액”은 법 제23조제3항 전단

In [75]:
## 리스트내포
lt = [i for i in range(11)]
lt

## 일반 for 문
lt2 = []
for i in range(11):
    lt2.append(i)

lt2


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [109]:

# ## 3.3. 점수 필터링 (예: 0.8 이상만 유효한 문서로 간주)

# threshold = 0.8
# filtered_docs = [doc for doc, score in retrieved_docs_with_score if score >= threshold]

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기 피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기 피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

prompt


"\n[identity]\n- 당신은 전세사기 피해 법률 전문가입니다.\n- [context]를 참고하여 사용자의 질문에 답변해주세요.\n- 전세사기 피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.\n\n[context]\n{retrieved_docs}\n\nQuestion: {query}\n"

In [111]:


## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=context,
    query=query,
)
formatted_prompt

"\n[identity]\n- 당신은 전세사기 피해 법률 전문가입니다.\n- [context]를 참고하여 사용자의 질문에 답변해주세요.\n- 전세사기 피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.\n\n[context]\n['전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\\n\\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\\n\\n\\n\\n국토교통부(피해지원총괄과) 044-201-5233, 5234\\n\\n\\n\\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\\n\\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\\n\\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\\n\\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\\n\\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\\n\\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\\n\\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\\n\\n    가. 다음의 계산식에 따라 계산한 금액\\n\\n  \\n\\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따

In [112]:

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='답변을 할 수 없습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 2105, 'total_tokens': 2112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BeuZPe6eK5at8iwYGyhPqajv9Px7J', 'finish_reason': 'stop', 'logprobs': None}, id='run--257d4992-54ac-47d7-98fd-16c01b4d37af-0', usage_metadata={'input_tokens': 2105, 'output_tokens': 7, 'total_tokens': 2112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})